In [310]:
import os
import pandas as pd
import re
from time import time  # To time our operations
from collections import defaultdict  # For word frequency
import spacy  # For preprocessing
import en_core_web_lg
from gensim.models.phrases import Phrases, Phraser
import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

import multiprocessing

from gensim.models import Word2Vec

In [311]:
nlp = en_core_web_lg.load()

In [312]:
import matplotlib.pyplot as plt
%matplotlib inline
 
import seaborn as sns
sns.set_style("darkgrid")

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [313]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Kacper\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [314]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [315]:
sid=SentimentIntensityAnalyzer()

In [316]:
def cleaning(doc):
    txt = [token.lemma_ for token in doc if not token.is_stop]
    if len(txt) > 2:
        return ' '.join(txt)

In [317]:
df=pd.DataFrame() #reset
df_clean=pd.DataFrame() #reset

In [318]:
from itertools import chain 

In [348]:
file=open('text_polsat.txt','r')
content =file.read()
content_list = content.split("- ")
cont_list=[]
for cont in content_list:
    cont_list.append(cont.split("\n\n"))

flat_list = list(chain.from_iterable(cont_list))

flat_list=[txt.lower().replace(" po ", " civic_platform ") for txt in flat_list]

file.close()

df=pd.DataFrame(flat_list)
#brief_cleaning = (re.sub("[^A-Za-z']+", ' ', str(row)).lower() for row in df[0])

#t = time()
#txt = [cleaning(doc) for doc in nlp.pipe(brief_cleaning, batch_size=5000, n_threads=-1)]
#print('Time to clean up everything: {} mins'.format(round((time() - t) / 60, 2)))

#df_clean = pd.DataFrame({'clean': txt})
#df_clean = df_clean.dropna().drop_duplicates()

In [349]:
flat_list[534]

'on friday, the sejm voted on nine senate amendments to the law on the solidarity allowance granted to counteract the negative effects of covid-19. president andrzej duda submitted the bill to the sejm.'

In [350]:
df_clean = df.dropna().drop_duplicates()
df_clean['sent']=df_clean.apply(lambda x: sent(x),axis=1)
df_clean['clean']=df_clean[0]

In [351]:
def sent(x):
    return sid.polarity_scores(x.iloc[0])

def tusk(x):
    if ' tusk ' in x.lower():
        return 1
    else:
        return 0
    
def kaczynski(x):
    if ' kaczynski ' in x.lower() or ' kaczy_ski ' in x.lower():
        return 1
    else:
        return 0
    
def pis(x):
    if ' law_justice ' in x.lower() or ' pis ' in x.lower():
        return 1
    else:
        return 0
    
def po(x):
    if ' civic_platform ' in x.lower():
        return 1
    else:
        return 0
    
def confederation(x):
    if ' confederation ' in x.lower():
        return 1
    else:
        return 0
    

In [352]:
def korwin(x):
    if 'korwin' in x.lower():
        return 1
    else:
        return 0

In [353]:
df_clean['sent']=df_clean.apply(lambda x: sent(x),axis=1)

In [354]:
df_clean['neg'] = df_clean['sent'].apply(lambda x: x.get('neg'))
df_clean['pos'] = df_clean['sent'].apply(lambda x: x.get('pos'))
df_clean['compound'] = df_clean['sent'].apply(lambda x: x.get('compound'))

In [355]:
df_clean['tusk'] = df_clean['clean'].apply(lambda x: tusk(x))
df_clean['tusk_emo']=df_clean['tusk']*df_clean['compound']
df_clean['kaczynski'] = df_clean['clean'].apply(lambda x: kaczynski(x))
df_clean['kaczynski_emo']=df_clean['kaczynski']*df_clean['compound']
df_clean['pis'] = df_clean['clean'].apply(lambda x: pis(x))
df_clean['pis_emo']=df_clean['pis']*df_clean['compound']
df_clean['po'] = df_clean['clean'].apply(lambda x: po(x))
df_clean['po_emo']=df_clean['po']*df_clean['compound']
df_clean['confederation'] = df_clean['clean'].apply(lambda x: confederation(x))
df_clean['confederation_emo']=df_clean['confederation']*df_clean['compound']
df_clean['korwin'] = df_clean['clean'].apply(lambda x: korwin(x))
df_clean['korwin_emo']=df_clean['korwin']*df_clean['compound']

In [356]:
print(df_clean['kaczynski_emo'].sum())
print(df_clean['tusk_emo'].sum())
print(df_clean['pis_emo'].sum())
print(df_clean['po_emo'].sum())
print(df_clean['confederation_emo'].sum())
print(df_clean['korwin_emo'].sum())

0.9728999999999999
-6.741900000000001
0.7028000000000025
12.9367
8.4462
-3.1227


In [343]:
df_clean[df_clean['korwin']==1][['clean','compound','pos','neg']].head(20)

,clean,compound,pos,neg
654,participated in the zn. the polish coalition c...,0.8313,0.028,0.011
761,janusz korwin-mikke said that it was a &quot;d...,0.0000,0.000,0.000
767,he added\n\nvideo: see the interview in the &q...,0.4939,0.110,0.000
1335,added radosław ślusarczyk from the pracownia n...,-0.0772,0.033,0.041
1419,emphasized kołodziejczak during the tuesday pr...,-0.4657,0.063,0.073
2335,terlecki said\n\nhe stated that the amendment ...,-0.8779,0.047,0.127
2855,"reported janusz korwin-mikke, who was an advis...",-0.8934,0.044,0.156
3175,"said janusz korwin-mikke, who has been shockin...",-0.7964,0.000,0.172
3179,said korwin-mikke.\n\nsee: braun delayed the p...,0.2960,0.086,0.053
3183,he said.\n\nvideo: korwin-mikke reveals shocki...,-0.7640,0.000,0.202


In [344]:
n=654

In [345]:
doc=nlp(df_clean.loc[n,0])

In [347]:
df_clean.loc[n,0]

'participated in the zn. the polish coalition club-psl-kukiz15 club was represented by around 15 mps, including psl president władysław kosiniak-kamysz, paweł kukiz and stanisław tyszka.\n\nsee: rainbow flags on the presidential oath. &quot;poland is for everyone&quot;\n\nthe head of the sejm circle of the confederation, jakub kulesza, announced that the session of the national assembly would be attended by mps janusz korwin-mikke, krzysztof bosak (former presidential candidate), grzegorz braun, artur dziambor, michał urbaniak, krystian kamiński and krzysztof tuduj.\n\nsee: the left mp approached the president. she handed him the letter\n\nformer presidents lech wałęsa and bronisław komorowski, former prime ministers ewa kopacz, leszek miller, włodzimierz cimoszewicz and former speaker of the sejm and former head of the ministry of foreign affairs radosław sikorski, former speaker of the sejm and former head of the civic platform grzegorz schetyna, did not take part in the national ass

In [346]:
print(df_clean.loc[n,0])
print(df_clean.loc[n,'compound'],' ',df_clean.loc[n,'pos'],' ',df_clean.loc[n,'neg'])

participated in the zn. the polish coalition club-psl-kukiz15 club was represented by around 15 mps, including psl president władysław kosiniak-kamysz, paweł kukiz and stanisław tyszka.

see: rainbow flags on the presidential oath. &quot;poland is for everyone&quot;

the head of the sejm circle of the confederation, jakub kulesza, announced that the session of the national assembly would be attended by mps janusz korwin-mikke, krzysztof bosak (former presidential candidate), grzegorz braun, artur dziambor, michał urbaniak, krystian kamiński and krzysztof tuduj.

see: the left mp approached the president. she handed him the letter

former presidents lech wałęsa and bronisław komorowski, former prime ministers ewa kopacz, leszek miller, włodzimierz cimoszewicz and former speaker of the sejm and former head of the ministry of foreign affairs radosław sikorski, former speaker of the sejm and former head of the civic platform grzegorz schetyna, did not take part in the national assembly, an

In [299]:
for token in doc:
    print(f"{token.text:15} {token.pos:10} {token.tag_:10} {token.dep_:10} {spacy.explain(token.tag_)}")

on                      85 IN         prep       conjunction, subordinating or preposition
the                     90 DT         det        determiner
basis                   92 NN         pobj       noun, singular or mass
of                      85 IN         prep       conjunction, subordinating or preposition
this                    90 DT         det        determiner
motion                  92 NN         pobj       noun, singular or mass
,                       97 ,          punct      punctuation mark, comma
the                     90 DT         det        determiner
civic_platform          96 NNP        nsubj      noun, proper singular
wants                  100 VBZ        ROOT       verb, 3rd person singular present
to                      94 TO         aux        infinitival "to"
recall                 100 VB         xcomp      verb, base form
jarosław                96 NNP        dobj       noun, proper singular
kaczyński               96 NNP        dobj       noun, proper sin

In [ ]:
sid.polarity_scores('mary loves john')

In [288]:
doc1=nlp('mary loves john')

In [289]:
doc2=nlp('john is loved by mary')

In [290]:
for token in doc1:
    print(f"{token.text:15} {token.pos:10} {token.tag_:10} {token.dep_:10} {spacy.explain(token.tag_)}")

mary                    96 NNP        nsubj      noun, proper singular
loves                  100 VBZ        ROOT       verb, 3rd person singular present
john                    96 NNP        dobj       noun, proper singular


In [291]:
for token in doc2:
    print(f"{token.text:15} {token.pos:10} {token.tag_:10} {token.dep_:10} {spacy.explain(token.tag_)}")

john                    96 NNP        nsubjpass  noun, proper singular
is                      87 VBZ        auxpass    verb, 3rd person singular present
loved                  100 VBN        ROOT       verb, past participle
by                      85 IN         agent      conjunction, subordinating or preposition
mary                    96 NNP        pobj       noun, proper singular
